In [6]:
import os

os.environ["DGLBACKEND"] = "pytorch"

import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F

# Loading Cora Dataset¶

In [7]:
dataset = dgl.data.CoraGraphDataset()
print(f"Number of categories: {dataset.num_classes}")

/Users/hoadinh/.dgl/cora_v2.zip: 100%|██████████| 132k/132k [00:00<00:00, 737kB/s] 


Extracting file to /Users/hoadinh/.dgl/cora_v2_d697a464
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Number of categories: 7


In [8]:
print(type(dataset))

<class 'dgl.data.citation_graph.CoraGraphDataset'>


In [9]:
g = dataset[0]
print(type(g))

<class 'dgl.heterograph.DGLGraph'>


A DGL graph can store node features and edge features in two dictionary-like attributes called ndata and edata. In the DGL Cora dataset, the graph contains the following node features:

* ```train_mask``` A boolean tensor indicating whether the node is in the training set.
* ```val_mask``` A boolean tensor indicating whether the node is in the validation set.
* ```test_mask``` A boolean tensor indicating whether the node is in the test set.
* ```label``` The ground truth node category.
* ```feat``` The node features.

In [10]:
print("Node features")
print(g.ndata)
print("Edge features")
print(g.edata)

Node features
{'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}
Edge features
{}


# Defining a Graph Convolutional Network (GCN)

In [11]:
from dgl.nn import GraphConv


class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Training the GCN

Training this GCN is similar to training other PyTorch neural networks.

In [16]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print(
                f"In epoch {e}, loss: {loss:.3f}, val acc: {val_acc:.3f} (best {best_val_acc:.3f}), test acc: {test_acc:.3f} (best {best_test_acc:.3f})"
            )


# Create the model with given dimensions
model = GCN(
    in_feats=g.ndata["feat"].shape[1], 
    h_feats=16, 
    num_classes=dataset.num_classes
)
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.136 (best 0.136), test acc: 0.135 (best 0.135)
In epoch 5, loss: 1.896, val acc: 0.426 (best 0.426), test acc: 0.453 (best 0.480)
In epoch 10, loss: 1.819, val acc: 0.670 (best 0.670), test acc: 0.689 (best 0.689)
In epoch 15, loss: 1.720, val acc: 0.714 (best 0.714), test acc: 0.720 (best 0.720)
In epoch 20, loss: 1.596, val acc: 0.732 (best 0.732), test acc: 0.745 (best 0.745)
In epoch 25, loss: 1.452, val acc: 0.730 (best 0.736), test acc: 0.757 (best 0.751)
In epoch 30, loss: 1.290, val acc: 0.740 (best 0.740), test acc: 0.760 (best 0.763)
In epoch 35, loss: 1.120, val acc: 0.744 (best 0.744), test acc: 0.756 (best 0.759)
In epoch 40, loss: 0.949, val acc: 0.752 (best 0.752), test acc: 0.765 (best 0.765)
In epoch 45, loss: 0.788, val acc: 0.760 (best 0.760), test acc: 0.770 (best 0.770)
In epoch 50, loss: 0.645, val acc: 0.768 (best 0.768), test acc: 0.773 (best 0.773)
In epoch 55, loss: 0.522, val acc: 0.774 (best 0.774), test acc: 0.776 (best 0